## 1. Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
import warnings

# 경고 메시지 무시 (데이터가 너무 짧거나 상수일 때 발생하는 경고 등)
warnings.filterwarnings('ignore')

## 2. 데이터 전처리

In [2]:
train = pd.read_csv('../open/train.csv')

# year, month, item_id 기준으로 value 합산 (seq만 다르다면 value 합산)
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

pivot.head()

ym,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,...,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
item_id,,,,,,,,,,,,,,,,,,,,,
AANGBULD,14276.0,52347.0,53549.0,0.0,26997.0,84489.0,0.0,0.0,0.0,0.0,...,428725.0,144248.0,26507.0,25691.0,25805.0,0.0,38441.0,0.0,441275.0,533478.0
AHMDUILJ,242705.0,120847.0,197317.0,126142.0,71730.0,149138.0,186617.0,169995.0,140547.0,89292.0,...,123085.0,143451.0,78649.0,125098.0,80404.0,157401.0,115509.0,127473.0,89479.0,101317.0
ANWUJOKX,0.0,0.0,0.0,63580.0,81670.0,26424.0,8470.0,0.0,0.0,80475.0,...,0.0,0.0,0.0,27980.0,0.0,0.0,0.0,0.0,0.0,0.0
APQGTRMF,383999.0,512813.0,217064.0,470398.0,539873.0,582317.0,759980.0,216019.0,537693.0,205326.0,...,683581.0,2147.0,0.0,25013.0,77.0,20741.0,2403.0,3543.0,32430.0,40608.0
ATLDMDBO,143097177.0,103568323.0,118403737.0,121873741.0,115024617.0,65716075.0,146216818.0,97552978.0,72341427.0,87454167.0,...,60276050.0,30160198.0,42613728.0,64451013.0,38667429.0,29354408.0,42450439.0,37136720.0,32181798.0,57090235.0


## 3. 공행성쌍 탐색
- 각 (A, B) 쌍에 대해 lag = 1 ~ max_lag까지 Pearson 상관계수 계산
- 절댓값이 가장 큰 상관계수와 lag를 선택
- |corr| >= corr_threshold이면 A→B 공행성 있다고 판단

In [3]:
def check_stationarity(series, p_threshold=0.05):
    """
    ADF Test를 통해 정상성 여부를 확인합니다.
    p-value가 임계값보다 낮으면 정상 시계열로 판단(True 반환).
    """
    try:
        result = adfuller(series, autolag='AIC')
        p_value = result[1]
        return p_value < p_threshold
    except:
        # 데이터가 너무 짧거나 분산이 0인 경우 등 예외 처리
        return False

def make_stationary(series):
    """
    비정상 시계열을 차분(Differencing)하여 정상성을 확보합니다.
    (여기서는 1차 차분만 시도하고, 그래도 안 되면 그대로 사용하거나 제외하는 로직)
    """
    if check_stationarity(series):
        return series, False # 이미 정상임
    
    diff_series = np.diff(series)
    # 차분 후 정상성 재확인 (엄격하게 하려면 여기서 check 다시 수행)
    return diff_series, True # 차분 적용됨

def find_granger_pairs(pivot, max_lag=6, min_nonzero=12, p_threshold=0.05):
    items = pivot.index.to_list()
    
    results = []

    # 진행 상황 표시를 위한 tqdm
    print("Granger Causality Test 진행 중...")
    
    for leader in tqdm(items):
        x_origin = pivot.loc[leader].values.astype(float)
        
        # 데이터 희소성 체크
        if np.count_nonzero(x_origin) < min_nonzero:
            continue
            
        # Leader 데이터 정상화 (필요 시 차분)
        x_stat, x_diff = make_stationary(x_origin)
        
        for follower in items:
            if leader == follower:
                continue
            
            y_origin = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y_origin) < min_nonzero:
                continue

            # Follower 데이터 정상화
            y_stat, y_diff = make_stationary(y_origin)
            
            # 데이터 길이 맞추기 (차분 수행 시 길이가 1 줄어듦)
            # 두 데이터 중 하나라도 차분되었다면, 길이를 맞춰줘야 함 (앞부분 trim)
            len_x = len(x_stat)
            len_y = len(y_stat)
            min_len = min(len_x, len_y)
            
            curr_x = x_stat[-min_len:]
            curr_y = y_stat[-min_len:]
            
            # Granger Test를 위한 데이터셋 구성 (T x 2)
            # 컬럼 순서: [Target(Follower), Predictor(Leader)] ★중요★
            data = np.column_stack([curr_y, curr_x])
            
            try:
                # max_lag까지 테스트 수행 (verbose=False로 로그 출력 끔)
                # 데이터 길이가 lag보다 충분히 커야 함
                if len(data) <= max_lag + 2:
                    continue

                gc_res = grangercausalitytests(data, max_lag, verbose=False)
                
                best_p_value = 1.0
                best_lag = 0
                best_f_score = 0.0
                
                # 1 ~ max_lag 결과를 순회하며 가장 유의한(p-value가 낮은) lag 찾기
                for lag in range(1, max_lag + 1):
                    # ssr_ftest의 p-value 추출 (index 1)
                    ftest_res = gc_res[lag][0]['ssr_ftest']
                    p_val = ftest_res[1]
                    f_score = ftest_res[0]
                    
                    if p_val < best_p_value:
                        best_p_value = p_val
                        best_lag = lag
                        best_f_score = f_score
                
                # 유의수준(p_threshold)보다 p-value가 낮으면 인과관계가 있다고 판단
                if best_p_value < p_threshold:
                    results.append({
                        "leading_item_id": leader,
                        "following_item_id": follower,
                        "best_lag": best_lag,
                        "min_p_value": best_p_value, # 상관계수 대신 p-value 저장
                        "f_score": best_f_score      # 참고용 F-score
                    })
                    
            except Exception as e:
                continue

    pairs = pd.DataFrame(results)
    return pairs

# 실행
pairs = find_granger_pairs(pivot, max_lag=4, p_threshold=0.05) 
# 주의: Granger Test는 연산 비용이 높으므로 max_lag를 너무 크게 잡거나 모든 쌍을 다 돌리면 오래 걸릴 수 있습니다.
# 필요하다면 앞단에서 상관계수로 1차 필터링을 한 후 Granger Test를 수행하는 Hybrid 방식도 고려 가능합니다.

print(f"탐색된 Granger 인과관계 쌍 수: {len(pairs)}")
if not pairs.empty:
    display(pairs.head().sort_values(by='min_p_value'))
else:
    print("조건을 만족하는 공행성 쌍이 없습니다. p_threshold를 높이거나 조건을 완화해보세요.")

Granger Causality Test 진행 중...


100%|██████████| 100/100 [00:32<00:00,  3.04it/s]

탐색된 Granger 인과관계 쌍 수: 1321


,leading_item_id,following_item_id,best_lag,min_p_value,f_score
2,AANGBULD,LLHREMKS,2,0.000774,8.853258
4,AANGBULD,LTOYKIML,1,0.004213,9.271432
3,AANGBULD,LSOIUSXD,1,0.013304,6.744140
1,AANGBULD,FWUCPMMW,1,0.014882,6.508835
0,AANGBULD,FTSVTTSR,3,0.022378,3.664208


## 4. 회귀 모델 학습
- 시계열 데이터 안에서 '한 달 뒤 총 무역량(value)을 맞추는 문제'로 self-supervised 학습
- 탐색된 모든 공행성쌍 (A,B)에 대해 월 t마다 학습 샘플 생성
- input X:
1) B_t (현재 총 무역량(value))
2) B_{t-1} (직전 달 총 무역량(value))
3) A_{t-lag} (lag 반영된 총 무역량(value))
4) max_corr, best_lag (관계 특성)
- target y:
1) B_{t+1} (다음 달 총 무역량(value))
- 이러한 모든 샘플을 합쳐 LinearRegression 회귀 모델을 학습

In [4]:
def build_training_data(pivot, pairs):
    """
    공행성쌍 + 시계열을 이용해 (X, y) 학습 데이터를 만드는 함수
    input X:
      - b_t, b_t_1, a_t_lag, min_p_value, best_lag
    target y:
      - b_t_plus_1
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        
        # [수정] max_corr 대신 min_p_value 사용
        p_val = float(row.min_p_value)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t+1이 존재하고, t-lag >= 0인 구간만 학습에 사용
        for t in range(max(lag, 1), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "min_p_value": p_val,  # [수정] 피처 이름 변경
                "best_lag": float(lag),
                "target": b_t_plus_1,
            })

    df_train = pd.DataFrame(rows)
    return df_train

df_train_model = build_training_data(pivot, pairs)
print('생성된 학습 데이터의 shape :', df_train_model.shape)
df_train_model.head()

생성된 학습 데이터의 shape : (52261, 6)


,b_t,b_t_1,a_t_lag,min_p_value,best_lag,target
0,179641.0,119514.0,14276.0,0.022378,3.0,241787.0
1,241787.0,179641.0,52347.0,0.022378,3.0,191752.0
2,191752.0,241787.0,53549.0,0.022378,3.0,46240.0
3,46240.0,191752.0,0.0,0.022378,3.0,124546.0
4,124546.0,46240.0,26997.0,0.022378,3.0,24568.0


In [5]:
# 회귀모델 학습
# [수정] max_corr -> min_p_value 로 피처 변경
feature_cols = ['b_t', 'b_t_1', 'a_t_lag', 'min_p_value', 'best_lag']

train_X = df_train_model[feature_cols].values
train_y = df_train_model["target"].values

reg = LinearRegression()
reg.fit(train_X, train_y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


## 5. 회귀 모델 추론 및 제출(submission) 파일 생성
- 탐색된 공행성 쌍에 대해 후행 품목(following_item_id)에 대한 2025년 8월 총 무역량(value) 예측

In [6]:
def predict(pivot, pairs, reg):
    months = pivot.columns.to_list()
    n_months = len(months)

    # 가장 마지막 두 달 index (2025-7, 2025-6)
    t_last = n_months - 1
    t_prev = n_months - 2

    preds = []

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        
        # [수정] max_corr -> min_p_value
        p_val = float(row.min_p_value)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t_last - lag 가 0 이상인 경우만 예측
        if t_last - lag < 0:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]

        # [수정] 입력 피처 순서 및 값 변경 (corr -> p_val)
        X_test = np.array([[b_t, b_t_1, a_t_lag, p_val, float(lag)]])
        y_pred = reg.predict(X_test)[0]

        # (후처리 1) 음수 예측 → 0으로 변환
        # (후처리 2) 소수점 → 정수 변환 (무역량은 정수 단위)
        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })

    df_pred = pd.DataFrame(preds)
    return df_pred

In [7]:
submission = predict(pivot, pairs, reg)
submission.head()

1321it [00:00, 16486.08it/s]


,leading_item_id,following_item_id,value
0,AANGBULD,FTSVTTSR,362130
1,AANGBULD,FWUCPMMW,187922
2,AANGBULD,LLHREMKS,172576
3,AANGBULD,LSOIUSXD,407552
4,AANGBULD,LTOYKIML,580145


In [8]:
submission.to_csv('./submissions/baseline_granger_causality.csv', index=False)